## 1. 打开浏览器


In [1]:
from Util.Selenium_Util import Selenium_Edge
edge_driver = Selenium_Edge(proxy=True)

下载目录:
F:\服创\我的项目\Chinese_Resume_analysis\NER相关


In [2]:
from Util import util
all_path = util.create_or_find_date_dir()
# edge_driver.get_driver('https://chat.openai.com/c/549532f5-8f84-49b6-ab58-46682fd7e3a3')

In [1]:
import os
for filename in os.listdir(all_path):
    # print(filename)
    major_path = os.path.join(all_path, filename)
    pdf_path = os.path.join(major_path,filename) + ".pdf"
    txt_path = os.path.join(major_path, filename) + ".txt"
    json_path = os.path.join(major_path, filename) + ".json"
    # print(file_pdf)
    if not os.path.exists(pdf_path):
        print("pdf不存在")
        continue
    if not os.path.exists(txt_path):
        text = util.get_str_from_pdf(pdf_path)
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"txt已生成于目录{txt_path}")
    if os.path.exists(json_path):
        print("json文件已存在")
        continue
    with open(json_path, 'w', encoding='utf-8') as f:
        f.write("")

NameError: name 'all_path' is not defined

In [2]:
import json
import os
all_tag_path = './Corpus_Path/all_tag.json'
key_mapping_path = './Corpus_Path/key_mapping.json'
with open(all_tag_path, 'r', encoding='utf-8')as f:
    all_tag = json.load(f)
with open(key_mapping_path, 'r', encoding='utf-8')as f:
    key_mapping = json.load(f)
all_tag,key_mapping

({'name': '',
  'bir': '',
  'age': '',
  'email': '',
  'gend': '',
  'tel': '',
  'acad': '',
  'nati': '',
  'live': '',
  'poli': '',
  'unv': '',
  'work_experience': [{'time': '', 'org': '', 'role': '', 'work_desc': ''}],
  'education': [{'time': '', 'school': '', 'degree': '', 'edu_desc': ''}],
  'project_experience': [{'time': '',
    'project': '',
    'role': '',
    'project_desc': ''}],
  'training_experience': [{'time': '', 'training': '', 'training_desc': ''}],
  'awards': [{'time': '', 'award': ''}],
  'self_assessment': '',
  'career_objective': '',
  'skills': '',
  'interests': '',
  'certifications': '',
  'language_skills': '',
  'publications': '',
  'philosophy': '',
  'major_courses': '',
  'self_introduction': '',
  'other': ''},
 {'name': 'NM',
  'bir': 'BR',
  'age': 'AG',
  'email': 'EM',
  'gend': 'GD',
  'tel': 'TL',
  'acad': 'AC',
  'nati': 'NT',
  'live': 'LV',
  'poli': 'PL',
  'unv': 'UV',
  'time': 'TM',
  'org': 'OG',
  'role': 'RL',
  'work_desc': '

In [1]:
from Util import util
from Util import File_Util
from pprint import pprint
import os

all_path = util.create_or_find_date_dir()
index = 0
for filename in os.listdir(all_path):
    # print(filename)
    major_path = os.path.join(all_path, filename)
    pdf_path = os.path.join(major_path,filename) + ".pdf"
    txt_path = os.path.join(major_path, filename) + ".txt"
    json_path = os.path.join(major_path, filename) + ".json"
    # print(file_pdf)
    if not os.path.exists(pdf_path):
        print("pdf不存在")
        continue
    text = File_Util.get_text_from_txt(txt_path,pdf_path)
    ner_json = File_Util.get_json_info(json_path)
    print(key_mapping)
    # 从标注好的json文件中映射原始文本索引
    json_index = util.marking_tag(ner_json,text,log_info=filename)
    pprint(json_index)
    # 标注好的json文件
    for key, value in ner_json.items():
        print(value)
    index = index + 1
    if index >= 1:
        break

ModuleNotFoundError: No module named 'File_Util'

In [7]:
def marking_tag(json_text,txt_text,key_mapping=key_mapping,log_info="101",is_idx=False):
    """
    打标记
    :param json_text: 原始标注好的json文件
    :param txt_text: 原始文本
    :param key_mapping: 简写映射
    :param log_info: 日志文本
    :return:
    """
    import re
    from Util.util import get_index_by_automaton,get_log_config,get_text_index
    from pprint import pprint
    log_config = get_log_config()
    all_tag_list = []
    # 遍历一级结构字典
    # pprint(json_text)
    for key, value in json_text.items():
        if isinstance(value, str):  # 如果字段值是字符串，则直接添加
            new_key = key_mapping[key]
            for split_tag in get_text_index(value,new_key,txt_text,log_info):
                all_tag_list.append(split_tag)
        elif isinstance(value, list):
            # 遍历二级结构列表
            for (index,item) in enumerate(value):
                # 遍历字典
                for sub_key, sub_value in item.items():
                    if is_idx:
                        # 带idx的
                        new_key = key_mapping[key]+"_"+key_mapping[sub_key]+"_"+str(index)
                    else:
                        # 不带idx的
                        new_key = key_mapping[key]+"_"+key_mapping[sub_key]
                    split_tag_list = get_text_index(sub_value,new_key,txt_text,log_info)
                    pprint(split_tag_list)
                    # print(index,item)
                    for split_tag in split_tag_list:
                        all_tag_list.append(split_tag)
    return all_tag_list
all_tag = marking_tag(ner_json,text)
from pprint import pprint
pprint(all_tag)

NameError: name 'get_text_index' is not defined

In [5]:
# 编码
# 初始化标注列表
tagged_text = ['O'] * len(text)

# 更新标注信息
for annotation in all_tag:
    for value, info in annotation.items():
        start, end, tag = info['start_index'], info['end_index'], info['tag']
        tagged_text[start] = 'B-' + tag  # 标记开始
        for i in range(start + 1, end + 1):
            tagged_text[i] = 'I-' + tag  # 标记内部

# 生成最终的标注结果
final_annotations = [[text[i], tagged_text[i]] for i in range(len(text))]
# 您来、不来我写作业了
# 打印结果
for item in final_annotations:
    print(f"{item[0]} {item[1]}")

# tag 代表没有标注第几次经历下标和标注类别名字
# tag_with_name 代表标注类别名字被标注
# tag_with_idx 代表标注第几次经历下标
# tag_wtih_idx_name 代表没有标注第几次经历下标和标注类别名字

with open ('./Corpus_Path/2024年03月03日15时01分/101/tag_with_idx.txt', 'w', encoding='utf-8') as f:
    for item in final_annotations:
        f.write(f"{item[0]} {item[1]}\n")

蔡 B-NM
于 I-NM
纬 I-NM
求 O
职 O
意 O
向 O
： O
市 B-CO
场 I-CO
专 I-CO
员 I-CO
年 O
龄 O
： O
2 B-AG
3 I-AG
岁 O
地 O
址 O
： O
广 B-LV
东 I-LV
省 I-LV
广 I-LV
州 I-LV
市 I-LV
电 O
话 O
： O
1 B-TL
3 I-TL
5 I-TL
1 I-TL
1 I-TL
2 I-TL
2 I-TL
3 I-TL
3 I-TL
4 I-TL
4 I-TL
邮 O
箱 O
： O
s B-EM
e I-EM
r I-EM
v I-EM
i I-EM
c I-EM
e I-EM
@ I-EM
5 I-EM
0 I-EM
0 I-EM
d I-EM
. I-EM
m I-EM
e I-EM
教 O
育 O
背 O
景 O
2 B-ED_TM
0 I-ED_TM
1 I-ED_TM
8 I-ED_TM
. I-ED_TM
9 I-ED_TM
- I-ED_TM
2 I-ED_TM
0 I-ED_TM
2 I-ED_TM
1 I-ED_TM
. I-ED_TM
6 I-ED_TM
华 B-ED_SC
南 I-ED_SC
师 I-ED_SC
范 I-ED_SC
大 I-ED_SC
学 I-ED_SC
附 I-ED_SC
属 I-ED_SC
中 I-ED_SC
学 I-ED_SC
高 B-ED_DG
中 I-ED_DG
主 B-ED_ES
修 I-ED_ES
课 I-ED_ES
程 I-ED_ES
基 B-ED_ES
本 I-ED_ES
会 I-ED_ES
计 I-ED_ES
、 O
统 B-ED_ES
计 I-ED_ES
学 I-ED_ES
、 O
市 B-ED_ES
场 I-ED_ES
营 I-ED_ES
销 I-ED_ES
、 O
国 B-ED_ES
际 I-ED_ES
市 B-ED_ES
场 I-ED_ES
营 I-ED_ES
销 I-ED_ES
、 O
市 B-ED_ES
场 I-ED_ES
调 I-ED_ES
查 I-ED_ES
与 I-ED_ES
预 I-ED_ES
测 I-ED_ES
、 O
商 B-ED_ES
业 I-ED_ES
心 I-ED_ES
理 I-ED_ES
学 I-ED_ES
、 O
广 B-ED_ES
告 I-ED_ES
学

In [7]:
def decode_tagged_text(text, tagged_text):
    decoded_annotations = []
    current_annotation = None
    current_tag = None

    for char, tag in zip(text, tagged_text):
        if tag.startswith('B-'):
            # 如果当前已有标注，则先保存当前标注
            if current_annotation is not None:
                decoded_annotations.append((current_annotation, current_tag))
            # 开始新的标注
            current_annotation = char
            current_tag = tag[2:]  # 移除 'B-' 前缀以获取标记
        elif tag.startswith('I-') and current_annotation is not None:
            # 如果当前字符是标注的一部分，则添加到当前标注
            current_annotation += char
        else:
            # 当前字符不属于任何标注，保存并重置当前标注（如果有的话）
            if current_annotation is not None:
                decoded_annotations.append((current_annotation, current_tag))
                current_annotation = None
                current_tag = None

    # 确保最后一个标注被添加
    if current_annotation is not None:
        decoded_annotations.append((current_annotation, current_tag))

    return decoded_annotations

# 示例文本和标注
text = "This is an example."
tagged_text = ['O', 'O', 'O', 'O', 'B-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'O', 'O', 'O', 'O', 'O']

# 解码标注
decoded_annotations = decode_tagged_text(text, tagged_text)

# 打印解码结果
for annotation, tag in decoded_annotations:
    print(f"Annotation: {annotation}, Tag: {tag}")


Annotation:  is an , Tag: TAG
